# Data Analysis Using LLM's
### How this tool works: -
1. Sending comments in batches of 2000 to gpt-4's 128k model to get initial topics which the comments are divided into  
2. The summaries for these batches are then consolidated into one JSON file with details that include comment themes, percentage of the theme present in the data, number of positive and negative comments for each topic
3. Save the generated JSON file which we can use for future comparisons
4. We then pass the comments again, along with the themes discovered in the first pass, to get sub-themes of each theme.
5. We save the sub-themes in a separate JSON for future reference.
6. Using the data generated we plot 3 graphs; 1. where the percentage of themes is plotted. 2. The number of positive and negative comments for each topic. 3. Sub-topics for each Topic
7. Finally, we can ask specific question to the data using PandasAI (Eg. How many customers in the survey speak French?)

## Setting up Azure LLM
We use gpt-4-1106-preview with 128k tokens as context

In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import csv

load_dotenv()


#AzureOpenAI setup
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    # api_type='azure',
    api_version='2023-05-15',  # this may change in the future
    timeout=20*60,  # 20 minutes
)

#To generate summaries from 1000's of comments
def generate_summary(prompt):
    response = client.chat.completions.create(
        temperature=0.0,
        model="gpt-4-1106-preview",
        messages=[
                {"role": "system", "content":"""You are a product manager.
                """},
                {"role": "user", "content": prompt },
            ]
    )

    return response.choices[0].message.content


#To consolidate different Summaries (The prompt "Business Analyst" makes summaries more concise and to the point.)
def consolidate_summaries(prompt):
    response = client.chat.completions.create(
    temperature=0.0,
    model="gpt-4-1106-preview",
    messages=[
            {"role": "system", "content":"""You are a Business Analyst.
            """},
            {"role": "user", "content": prompt },
        ]
    )
    return response.choices[0].message.content



## Reading from the csv
We can change the name of the CSV and the column name that we want to analyse over here.

In [ ]:
column_name = "Q2 - Comment - Recommendations"  #We can extract the column we want to analyse by providing the appropriate name.
with open("survey.csv", 'r', newline='') as csvfile: #We can the input file here.
    reader = csv.DictReader(csvfile)
    if column_name not in reader.fieldnames:
        print("Field name not found!")
    column_data = [row[column_name] for row in reader if row[column_name].strip()]

print(f"Number of comments in the column: {len(column_data)}")

#Making batches of 2000 comments
batch_size = 2000 #We can change the number of comments in each batch, depending on the survey type.
total_batches = (len(column_data) + batch_size - 1)//batch_size
print(f"Number of batches: {total_batches}")


## Analyzing comments in batches to establish Themes

In [ ]:
appended_responses = []

for batch_number in range(total_batches):
    start_index = batch_number * batch_size
    end_index = (batch_number + 1) * batch_size
    comment_batch = column_data[start_index:end_index]
    prompt = """As a poduct manager, analyze if the comments seperated by a single quotes are positive or negative and give the weight by occurence of each topic % 
    give a summary in the following JSON format.
        {
    "date": "Todays_Date",
    "theme_weights": {
        "Theme 1": [{theme1_percentage}%, number of positive comments, number of negative comments],",
        "Theme 2": [{theme2_percentage}%, number of positive comments, number of negative comments]%",
        "Theme 3": [{theme3_percentage}%, number of positive comments, number of negative comments]%
    }
    "Summary": "Consolidated Summary of all the comments"
    }
    comments:""" + f"```{comment_batch}```"
    summary = generate_summary(prompt)
    print(f"######Summary of batch number {batch_number}: -")
    print(summary)
    appended_responses.append(summary)

## Consolidating different responses recieved from the LLM

In [ ]:
prompt3 = """
As a business Analyst, 
analyze the following summaries seperated by single quotes and give the consolidated weight by occurence percentage(%), number of positive comments and number of negative commenst for every theme along with a comprehensive summary,
Generate a JSON file in the following format:

{
  "date": "Todays_Date",
  "theme_weights": {
      "Theme 1": [{theme1_percentage}%, {number of positive comments}, {number of negative comments}],",
      "Theme 2": [{theme2_percentage}%, {number of positive comments}, {number of negative comments}]",
      "Theme 3": [{theme3_percentage}%, {number of positive comments}, {number of negative comments}]
  }
  "Summary": "Consolidated Summary of all the comments"
}
""" + f"Seperate Summaries:```{appended_responses}```"

#Comparing the summaries and generating a final summary
final_analysis = consolidate_summaries(prompt3)
print(final_analysis)

## Extracting the JSON file containing Main Topics with +ve & -ve comments 
It is saved as "analysis_<todays_date>.json"

In [ ]:
import json
from datetime import date

start_index = final_analysis.find('{')
end_index = final_analysis.rfind('}')

if start_index != -1 and end_index != -1:
    json_part = final_analysis[start_index:end_index+1]
    print(json_part)
    try:
        # Load the JSON part using a JSON parser
        json_data = json.loads(json_part)
        json_data["date"] = str(date.today()) #GPT can't get dates right :(
        print("Extracted JSON:", json_data)
        file_path = f"analysis_{str(date.today())}.json"
        with open(file_path, 'w') as json_file:
            json.dump(json_data, json_file, indent=4)
    except json.JSONDecodeError:
        print("Invalid JSON structure.")

        

## Second Pass to find subtopics under Initial themes

In [ ]:
appended_responses = []

for batch_number in range(total_batches):
    start_index = batch_number * batch_size
    end_index = (batch_number + 1) * batch_size
    comment_batch = column_data[start_index:end_index]
    prompt = """As a poduct manager, analyze the comments and subdivide them into categories along with weighted percentage of occurance, based on the initial analysis. 
    give a summary in the following JSON format.
        {
    "date": "Todays_Date",
    "theme_weights": {
        "Theme 1": [category 1, percentage occurance of category 1, category 2 percentage occurance of category 2, category 3, percentage occurance of category 3],",
        "Theme 2": [category 1, percentage occurance of category 1, category 2, percentage occurance of category 2, category 3, percentage occurance of category 3]",
        "Theme 3": [category 1, percentage occurance of category 1, category 2, percentage occurance of category 2, category 3, percentage occurance of category 3]
    }
    "Summary": "Consolidated Summary of all the comments"
    }
    comments:""" + f"```{comment_batch}```" + f"Previous Summary: {json_data}"
    summary = generate_summary(prompt)
    print(f"######Summary of batch number {batch_number}: -")
    print(summary)
    appended_responses.append(summary)
    
    

In [ ]:
prompt4 = """
As a business Analyst, 
analyze the following summaries seperated by triple backticks and give the complete list of subtopics for each them. Include a comprehensive summary,
Generate a JSON file in the following format:

{
  "date": "Todays_Date",
  "theme_weights": {
        "Theme 1": [category 1, percentage occurance of category 1, category 2 percentage occurance of category 2, category 3, percentage occurance of category 3],",
        "Theme 2": [category 1, percentage occurance of category 1, category 2, percentage occurance of category 2, category 3, percentage occurance of category 3]",
        "Theme 3": [category 1, percentage occurance of category 1, category 2, percentage occurance of category 2, category 3, percentage occurance of category 3]
  }
  "Summary": "Consolidated Summary of all the comments"
}
""" + f"Seperate Summaries:```{appended_responses}```"

#Comparing the summaries and generating a final summary
final_themes = consolidate_summaries(prompt4)
print(final_themes)

In [ ]:
start_index = final_themes.find('{')
end_index = final_themes.rfind('}')

if start_index != -1 and end_index != -1:
    json_part = final_themes[start_index:end_index+1]
    print(json_part)
    try:
        # Load the JSON part using a JSON parser
        theme_json_data = json.loads(json_part)
        theme_json_data["date"] = str(date.today()) #GPT can't get dates right :(
        print("Extracted JSON:", theme_json_data)
        file_path = f"subtopic_analysis_{str(date.today())}.json"
        with open(file_path, 'w') as json_file:
            json.dump(json_data, json_file, indent=4)
    except json.JSONDecodeError:
        print("Invalid JSON structure.")

## Plotting the data for percentage of occurance of themes 
Here we can see the most common topics that are mentioned in the survey along with an overall summary. 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6)) #Setting the figure size in inches

# Extracting data for plotting
date = json_data["date"]
theme_weights = json_data["theme_weights"]

themes = list(theme_weights.keys())
percentages = [float(weight[0][:-1]) for weight in theme_weights.values()]

# Plotting the histogram
plt.bar(themes, percentages, color='blue')
plt.xticks(rotation=45, ha='right') #Rotating and aligning the x-axis labels
plt.xlabel('Comment Themes')
plt.ylabel('Percentage')
plt.title(f'Most Common Comments on {date}')
plt.tight_layout()  # Adjusts the layout
plt.savefig('bar_plot.png', dpi=300)  # saves the plot as 'bar_plot.png' with the specified dpi
plt.show()

print(f"Executive Summary: {json_data['Summary']}")



## Plotting the Positive and Negative comments for each Topic

In [ ]:
import numpy as np

# Extracting data for plotting
themes = list(theme_weights.keys())
positive_comments = [weight[1] for weight in theme_weights.values()]
negative_comments = [-weight[2] for weight in theme_weights.values()]

pos = np.arange(len(themes))
width = 0.4

# Plotting the data
plt.figure(figsize=(10, 6))
plt.bar(pos - width/2, positive_comments, width, color='green', label='Positive Comments')
plt.bar(pos + width/2, negative_comments, width, color='red', label='Negative Comments')
plt.xticks(pos, themes, rotation=45, ha='right')
plt.xlabel('Themes')
plt.ylabel('Number of Comments')
plt.title('Theme Weights')
plt.legend()

plt.tight_layout()  # Adjusts the layout
plt.savefig('comment_distribution.png', dpi=300)  # saves the plot as 'theme_weights.png' with a resolution of 300 dpi
plt.show()

## Plotting Weights of Sub-Topics for Each Major Theme

In [ ]:

# Convert the data into a suitable format
subtopic_name = 'Call Quality'  #We can change the subtopic name here to get the subtopics for that theme.

# Extracting the subtopic data for a specific theme
subtopic_data = theme_json_data['theme_weights'][subtopic_name] 

categories = {}
for i in range(0, len(subtopic_data), 2):
    category = subtopic_data[i]
    value = subtopic_data[i+1]
    if isinstance(value, str) and '%' in value:  # Check if value is a percentage
        value = float(value.replace('%', ''))  # Remove '%' and convert to float
    else:
        value = float(value)  # Convert to float
    categories[category] = value

labels = list(categories.keys())
values = list(categories.values())

plt.barh(labels, values, color='blue')
plt.xlabel('Values')
plt.title(subtopic_name)
plt.show()

## Comparing with previous data stored as JSON.

In [ ]:
#Comparing json_data with the previous data

#Reading data from JSON file
with open('analysis_2023-04-14.json') as json_file: #We can change the file name here to the file we wish to compare the data with.
    old_data = json.load(json_file)

#Extracting data for plotting
old_date = old_data["date"]
old_theme_weights = old_data["theme_weights"]

old_themes = list(old_theme_weights.keys())
old_percentages = [float(weight[0][:-1]) for weight in old_theme_weights.values()]

#comparison of the data
plt.figure(figsize=(10, 6)) #Setting the figure size in inches
plt.plot(themes, percentages,label='Data of current Summaries', color='blue')
plt.plot(old_themes, old_percentages, label='Data of previous Summaries', color='red') #new percentages
plt.xticks(rotation=45, ha='right') #Rotating and aligning the x-axis labels
plt.xlabel('Comment Themes')
plt.ylabel('Percentage')
plt.title(f'Most Common Comments on {date} and {old_date}')
plt.legend()
plt.tight_layout()  # Adjusts the layout
plt.show()

# Pandas-AI experimentation

In [26]:
import pandas as pd
from pandasai import SmartDataframe
from pandasai.llm import AzureOpenAI

llm = AzureOpenAI(
    api_token=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version="2023-05-15",
    deployment_name="gpt-4-1106-preview"
)

df = SmartDataframe("survey.csv", config={'llm': llm}) #Loading the survey data here

## Can ask specific question to the data
For more granularity we can ask exact question about the data using PandasAI. Some Exmple queries are: -
1. Give me the number of non-english speakers in the surver
2. Give me a pie chart of the distribution of languages customers speak in the survey data with a clear legend
3. How many customers were present in the survey.

In [ ]:
response = df.chat("How many people speak French in the Survey?") #We can change the question here to the one we want to ask.
print(response)